<h4> Finding Fraud

<h4> Look at these items - 

Email Count

Title length

English syntax

Abandoned Apps

In [1]:
import findspark
findspark.init('/opt/cloudera/parcels/CDH/lib/spark')
import sys
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import desc,col
import pyspark.sql.functions as f
from common_functions import time_log_start, time_log_finish
time_log_start()
job_name = 'Hive_Test'
conf = (
    SparkConf()
        .setAppName(job_name)
		.set("hive.metastore.uris","thrift://bdr3-itep-1.uspto.gov:9083")
		.set("spark.sql.extensions","com.qubole.spark.hiveacid.HiveAcidAutoConvertExtension")
		.set("spark.datasource.hive.warehouse.read.via.llap","false")
		.set("spark.sql.hive.llap","true")
		.set("spark.sql.hive.hwc.execution.mode","spark")
		.set("spark.kryo.registrator","com.qubole.spark.hiveacid.util.HiveAcidKyroRegistrator")
		.set("ssl","True")
		.set("spark.datasource.hive.warehouse.read.jdbc.mode","cluster")
		.set("spark.security.credentials.hiveserver2.enabled","false")
		.set("pyFiles","/opt/cloudera/parcels/CDH/lib/hive_warehouse_connector/pyspark_hwc-1.0.0.7.1.5.0-257.zip")
		.set("spark.jars","/opt/cloudera/parcels/CDH/lib/hive_warehouse_connector/hive-warehouse-connector-assembly-1.0.0.7.1.5.0-257.jar")  
        .set('spark.hadoop.hive.metastore.sasl.enabled','true')
    )
spark = (SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate())
all_databases_df = spark.sql("show databases")
all_databases_df.show()
time_log_finish()

Start of date and time of this section of code
2022-07-07 16:11:59
+------------------+
|      databaseName|
+------------------+
|               bdr|
|              csam|
|           default|
|               drp|
|                ds|
|information_schema|
|       patent_data|
|              ptab|
|               sys|
|              tept|
|                tm|
|               tqr|
|       tqr_archive|
|               trm|
|    trm_databridge|
|      trm_eogadmin|
|    trm_proceeding|
|     trm_tmngfpepp|
|      trm_tmngidmp|
+------------------+

Finish of date and time of this section of code
2022-07-07 16:12:22


<h4> Listing the Patent Fraud tables

In [2]:
spark.sql("show tables in patent_data").show(n=75,truncate=False)

+-----------+-------------------------------+-----------+
|database   |tableName                      |isTemporary|
+-----------+-------------------------------+-----------+
|patent_data|application_general_information|false      |
|patent_data|opsg_email_address             |false      |
|patent_data|opsg_physical_address          |false      |
|patent_data|opsg_telecommunication_details |false      |
|patent_data|palm_applications              |false      |
|patent_data|publicapplicationstatus        |false      |
|patent_data|suhee_applications             |false      |
+-----------+-------------------------------+-----------+



<h4> Pulling in the Patent data - Application General Information

In [3]:
time_log_start()
df_0 = spark.sql("Select * from patent_data.application_general_information")
print(f'{df_0.count():,}')
df_0.printSchema()
time_log_finish()

Start of date and time of this section of code
2022-07-07 16:12:23
1,176,873
root
 |-- applicationnumbertext: string (nullable = true)
 |-- abandoneddate: long (nullable = true)
 |-- applicationconfirmationnumber: string (nullable = true)
 |-- applicationfilecategory: string (nullable = true)
 |-- applicationfilereference: string (nullable = true)
 |-- applicationstatusdate: long (nullable = true)
 |-- applicationstatusdescriptiontext: string (nullable = true)
 |-- applicationtypecategory: string (nullable = true)
 |-- applicationwithdrawndate: long (nullable = true)
 |-- currentapplclass: string (nullable = true)
 |-- customernumbertext: string (nullable = true)
 |-- disposalindicator: string (nullable = true)
 |-- effectivefilingdate: long (nullable = true)
 |-- filingdate: long (nullable = true)
 |-- firstactiononmeritindicator: string (nullable = true)
 |-- firstinventortofileindicator: string (nullable = true)
 |-- grantdate: long (nullable = true)
 |-- groupartunitnumber: string 

In [4]:
#Selecting only a few fields of data

df_1 = df_0.select('applicationnumbertext','effectivefilingdate','inventiontitle','applicationstatusdescriptiontext').filter(col("inventiontitle").isNotNull())
df_1.show(n=30,truncate=False)

+---------------------+-------------------+------------------------------------------------------------------------+-----------------------------------------------------+
|applicationnumbertext|effectivefilingdate|inventiontitle                                                          |applicationstatusdescriptiontext                     |
+---------------------+-------------------+------------------------------------------------------------------------+-----------------------------------------------------+
|29000002             |718257600          |LT LAPMATE COMPUTER STAND                                               |Abandoned  --  Failure to Respond to an Office Action|
|29000005             |717998400          |BAY WINDOW                                                              |Abandoned  --  Failure to Respond to an Office Action|
|29000010             |717998400          |END MEMBER FOR A TUBE                                                   |Patented Case                

In [5]:
## Finding the counts of application status
df_1.groupby("applicationstatusdescriptiontext").count().sort(desc("count")).show(n=13,truncate=False)

+----------------------------------------------------------------------------+------+
|applicationstatusdescriptiontext                                            |count |
+----------------------------------------------------------------------------+------+
|Patented Case                                                               |597168|
|Provisional Application Expired                                             |180706|
|Application Dispatched from Preexam, Not Yet Docketed                       |144726|
|Abandoned  --  Failure to Respond to an Office Action                       |79314 |
|Docketed New Case - Ready for Examination                                   |66893 |
|Abandoned  --  Incomplete Application (Pre-examination)                     |38844 |
|Abandoned  --  Failure to Pay Issue Fee                                     |29723 |
|Application Undergoing Preexam Processing                                   |9728  |
|Non Final Action Mailed                              

<h3> Subsetting the dataset by "applicationstatusdescriptiontext" field

In [6]:
df_titles = df_1.filter(col("applicationstatusdescriptiontext").contains("Abandoned"))
print('Number of Abandoned records = ',f'{df_titles.count():,}')

Number of Abandoned records =  153,856


<h3> Finding the length of the invention title

<h4>And the language of the invention title

In [7]:
#from pynlp.pretrained import PretrainedPipeline
#language_detector_pipeline = PretrainedPipeline('detect_language_21', lang='xx')


In [8]:
#import pyspark.sql.functions as f
df_titles=df_titles.withColumn('title_length',f.length('inventiontitle'))
df_titles_length=df_titles.groupby("applicationnumbertext",'title_length').count().sort(desc("count"))
df_titles_length.sort(desc('title_length')).show(n=25)

+---------------------+------------+-----+
|applicationnumbertext|title_length|count|
+---------------------+------------+-----+
|             63207312|         600|    1|
|             63258405|         600|    1|
|             63222990|         600|    1|
|             63191359|         600|    1|
|             63033172|         600|    1|
|             29175989|         600|    1|
|             29175988|         599|    1|
|             63205724|         598|    1|
|             63201885|         577|    1|
|             63259054|         549|    1|
|             63164110|         506|    1|
|             63103044|         506|    1|
|             63120744|         500|    1|
|             63120749|         500|    1|
|             29779251|         500|    1|
|             63237837|         500|    1|
|             63120743|         500|    1|
|             63083049|         500|    1|
|             63091811|         500|    1|
|             63135651|         500|    1|
|          

<h3> Adding back in the invention titles and count of characters of the titles

In [9]:
df_titles_2 = df_titles_length.join(df_titles.select("applicationnumbertext","inventiontitle"), on='applicationnumbertext',how='left')
df_titles_2 = df_titles_2.withColumnRenamed("count","character_count_title")
df_titles_2.sort(desc("title_length")).show(n=13, truncate=False)

+---------------------+------------+---------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|applicationnumbertext|title_length|character_count_title|inventiontitle                                                                                                                                                                                                                                                                            

<h3> Getting Emails 

In [10]:
from pyspark.sql.functions import desc, col
df_1a = spark.sql("Select * from patent_data.opsg_email_address")
df_1b = df_1a.filter(col("emailaddresstext").isNotNull())
print(f'{df_1b.count():,}')
print(df_1b.printSchema())
df_email_groupby=df_1b.groupby("emailaddresstext","applicationnumbertext").count().sort(desc("count"))
df_email_groupby.show(n=9,truncate=False)

2,100,664
root
 |-- applicationnumbertext: string (nullable = true)
 |-- emailaddresstext: string (nullable = true)
 |-- deleteexistingemail: boolean (nullable = true)

None
+-----------------------------+---------------------+-----+
|emailaddresstext             |applicationnumbertext|count|
+-----------------------------+---------------------+-----+
|scott@bigskycomposites.com   |29445770             |5    |
|info@24ipusa.com             |29249735             |4    |
|info@24ipusa.com             |29584784             |4    |
|info@24ipusa.com             |29629185             |4    |
|info@24ipusa.com             |29345025             |4    |
|info@24ipusa.com             |29265222             |4    |
|jonathan.perry@acmeglobes.com|29666727             |4    |
|info@24ipusa.com             |29399664             |4    |
|info@24ipusa.com             |29552832             |4    |
+-----------------------------+---------------------+-----+
only showing top 9 rows



<h4>Emails counts - starting a "good" list of email extensions, a one of the filters in our decision tree

In [11]:
df_email_only_groupby=df_1b.groupby("emailaddresstext").count().sort(desc("count"))
df_email_only_groupby.show(n=33,truncate=False)

+-----------------------------------------+-----+
|emailaddresstext                         |count|
+-----------------------------------------+-----+
|eofficeaction@appcoll.com                |46704|
|USPTO@dockettrak.com                     |24302|
|e-office@sternekessler.com               |11852|
|mailroom@bskb.com                        |10355|
|ipdocketing@foley.com                    |10259|
|nike_docketing@cardinal-ip.com           |10209|
|efiling@knobbe.com                       |9799 |
|jayna.cartee@knobbe.com                  |9738 |
|PTO-125760@bannerwitcoff.com             |8937 |
|pair@jumpy-ip.com                        |8741 |
|uspto@dockettrak.com                     |8710 |
|usptopatentmail@cantorcolburn.com        |8484 |
|PPROCESSING@SUGHRUE.COM                  |8230 |
|USPTO@sughrue.com                        |8208 |
|KTSDocketing2@kilpatrick.foundationip.com|8170 |
|ipefiling@kilpatricktownsend.com         |8150 |
|pair_nixon@firsttofile.com               |7593 |


<h4> Matching the "Abandoned" application status with the email addresses

In [12]:
df_matched_emails = df_titles.join(df_1b, on='applicationnumbertext', how='inner')
print('Number of records = ', f'{df_matched_emails.count():,}')
df_matched_emails.printSchema()

Number of records =  170,798
root
 |-- applicationnumbertext: string (nullable = true)
 |-- effectivefilingdate: long (nullable = true)
 |-- inventiontitle: string (nullable = true)
 |-- applicationstatusdescriptiontext: string (nullable = true)
 |-- title_length: integer (nullable = true)
 |-- emailaddresstext: string (nullable = true)
 |-- deleteexistingemail: boolean (nullable = true)



<h3> Counting the Email addresses

In [13]:
df_signal_2=df_matched_emails.groupby('emailaddresstext').agg(f.count('emailaddresstext').alias("Count_of_emails"),f.max('title_length').alias('max_title_length'))
df_signal_2.where(col("Count_of_emails") > 6).sort(desc('max_title_length')).show(n=35,truncate=False)

+----------------------------+---------------+----------------+
|emailaddresstext            |Count_of_emails|max_title_length|
+----------------------------+---------------+----------------+
|realsuheepark@gmail.com     |521            |500             |
|XOXOAB@YAHOO.COM            |1406           |500             |
|lindblad2716@aol.com        |38             |500             |
|xoxoab@yahoo.com            |93             |498             |
|JOEMARTINTIGER@GMAIL.COM    |10             |415             |
|steven.a.nielsen@gmail.com  |95             |393             |
|Steve@NielsenPatents.com    |95             |393             |
|accelinflation@yahoo.com    |34             |382             |
|Alexanthony1984@hotmail.com |18             |359             |
|Mail@Patents.gs             |12             |307             |
|SethNehrbass@patents.gs     |12             |307             |
|SNehrbass@gmail.com         |12             |307             |
|sealsrosesandstars@yahoo.com|53        

<h4>  Looking at the invention titles

In [14]:
df_matched_emails.filter(col("emailaddresstext").contains('HoxiePatentMail@gmail.com')).show(truncate=False,vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 applicationnumbertext            | 63142435                                                                                                                                                                                                                                                                
 effectivefilingdate              | 1611723600                                                                                                                                                                                                                                                              
 inventiontitle                   | SYSTEMS AND METHODS FOR DNA ATTACHMENT BETWEEN NANOPORES     

In [15]:
df_matched_emails.filter(col("emailaddresstext").contains('eofficeaction@appcoll.com')).show(truncate=False,vertical=True)

-RECORD 0--------------------------------------------------------------------------------------
 applicationnumbertext            | 29321136                                                   
 effectivefilingdate              | 1215748800                                                 
 inventiontitle                   | TOOL HOOK                                                  
 applicationstatusdescriptiontext | Abandoned  --  Failure to Pay Issue Fee                    
 title_length                     | 9                                                          
 emailaddresstext                 | eofficeaction@appcoll.com                                  
 deleteexistingemail              | false                                                      
-RECORD 1--------------------------------------------------------------------------------------
 applicationnumbertext            | 29369623                                                   
 effectivefilingdate              | 1284

In [16]:
df_matched_emails.filter(col("emailaddresstext").contains('Alexanthony1984@hotmail.com')).show(truncate=False,vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 applicationnumbertext            | 63155038                                                                                                                                                                                                                                                                                                                                                                
 effectivefilingdate              | 1614574800                                                                                                                                                                

In [17]:
df_matched_emails.filter(col("emailaddresstext").contains('accelinflation@yahoo.com')).show(truncate=False,vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 applicationnumbertext            | 29511714                                                                                                                                                                                                          
 effectivefilingdate              | 1418360400                                                                                                                                                                                                        
 inventiontitle                   | 10,000 COLORED BUDS AND BULBS TEACHING ABOUT COLORS, 20,000 NEW CHEMICAL ELEMENTS                                                                                                                                 
 application

In [18]:
df_matched_emails.filter(col("emailaddresstext").contains('steven.a.nielsen@gmail.com')).show(truncate=False,vertical=True) 

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 applicationnumbertext            | 29578319                                                                                                                                                                                                                                                                                                                                                                                                  
 effectivefilingdate              | 1474344000                                                                                            

In [19]:
df_matched_emails.filter(col("emailaddresstext").contains('lindblad2716@aol.com')).show(truncate=False,vertical=True) 

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 applicationnumbertext            | 63256568                                                                                                                                                                                                                                                                                                                                             
 effectivefilingdate              | 1634356800                                                                                                                                                                                                      

In [20]:
df_matched_emails.filter(col("emailaddresstext").contains("costelloash4@outlook.com")).show(truncate=False,vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 applicationnumbertext            | 63191359                                                                                                                                                                                                                                                                                                                              

In [21]:
df_matched_emails.groupby("emailaddresstext").count().sort(desc("count")).show(n=35,truncate=False)



+---------------------------------+-----+
|emailaddresstext                 |count|
+---------------------------------+-----+
|eofficeaction@appcoll.com        |2746 |
|USPTO@dockettrak.com             |2408 |
|XOXOAB@YAHOO.COM                 |1406 |
|uspto@dockettrak.com             |826  |
|eofficeaction@bannerwitcoff.com  |824  |
|efiling@knobbe.com               |708  |
|jayna.cartee@knobbe.com          |705  |
|GPD@bannerwitcoff.com            |671  |
|centraldocket.im@pg.com          |597  |
|pair_pg@firsttofile.com          |597  |
|mayer.jk@pg.com                  |597  |
|pair_nixon@firsttofile.com       |571  |
|nike_docketing@cardinal-ip.com   |568  |
|x@xyztem.com                     |545  |
|MAILROOM@MG-IP.COM               |529  |
|e-office@sternekessler.com       |521  |
|realsuheepark@gmail.com          |521  |
|pair@jumpy-ip.com                |500  |
|56029@QUICKPATENTS.COM           |494  |
|patents@dbllawyers.com           |489  |
|pair@sinorica.com                

<h4> Matching email address against the length of invention title

<h4> Looking at the low number of applicaitons

In [22]:
df_email_only_groupby.filter(col("count") < 100).show(n=33,truncate=False) 

+----------------------------------+-----+
|emailaddresstext                  |count|
+----------------------------------+-----+
|ip.lhlaw@live.com                 |99   |
|rich@newmanlawlv.com              |99   |
|uspto@lhlaw.com                   |99   |
|uspto@zellerip.com                |99   |
|rdc@chabotlaw.com                 |99   |
|ARRIS.docketing@commscope.com     |99   |
|vladimir.skliba@morganlewis.com   |99   |
|pat@rogowskilaw.com               |99   |
|patents@newmanlawlv.com           |99   |
|PATENT@BOSELAW.COM                |99   |
|baker@gdllawfirm.com              |99   |
|patents@sri.com                   |99   |
|docketing@houstonllp.com          |99   |
|jlongmuir@adelphia.net            |99   |
|jlongmuir@roadrunner.com          |99   |
|kyle@zellerip.com                 |99   |
|ip.lhlaw@gmail.com                |99   |
|ipdocketing@ipllfirm.com          |99   |
|patent@insulet.com                |98   |
|dprincipe@phillipslytle.com       |98   |
|jwchilders

In [23]:
df_email_only_groupby.filter(col("emailaddresstext").contains("@yahoo")).show(n=33,truncate=False)

+-----------------------------+-----+
|emailaddresstext             |count|
+-----------------------------+-----+
|ptoactions@yahoo.com         |5686 |
|wangxuzhuo@yahoo.com         |884  |
|crossleypatentlaw@yahoo.com  |771  |
|markacrossley@yahoo.com      |771  |
|cgolaw@yahoo.com             |668  |
|steven_earl_shapiro@yahoo.com|316  |
|abbeyroad1126@yahoo.com      |178  |
|lyman.moulton@yahoo.com      |177  |
|emotion0002@yahoo.com        |165  |
|thomasinelectric@yahoo.com   |154  |
|frischknechtp@yahoo.com      |151  |
|wong.willy@yahoo.com         |142  |
|xoxoab@yahoo.com             |95   |
|cbmpatent@yahoo.com          |95   |
|reddyd55@yahoo.com           |91   |
|valerie365@yahoo.com         |87   |
|tmdjoseph@yahoo.com          |83   |
|mgibb@yahoo.com              |79   |
|sarahrhoades@yahoo.com       |77   |
|phil_virga@yahoo.com         |75   |
|saritoc@yahoo.com            |70   |
|theodorebaker@yahoo.com      |69   |
|p_a_su@yahoo.com             |69   |
|patentblog@

<h4> Matching up the emails with the titles

In [24]:
df_all = df_email_groupby.join(df_titles_2, on='applicationnumbertext', how='inner')
df_all.select("emailaddresstext","applicationnumbertext","count","title_length","inventiontitle").sort(desc("count")).show(n=13,truncate=False)

+----------------------------+---------------------+-----+------------+-------------------------------------------+
|emailaddresstext            |applicationnumbertext|count|title_length|inventiontitle                             |
+----------------------------+---------------------+-----+------------+-------------------------------------------+
|scott@bigskycomposites.com  |29445770             |5    |16          |BUNG/TOTE WRENCH                           |
|info@24ipusa.com            |29720318             |3    |24          |Roofing Material Display                   |
|info@24ipusa.com            |29246065             |3    |10          |SEAT COVER                                 |
|ipdocket@thompsoncoburn.com |29172966             |3    |4           |DESK                                       |
|devolpatents@gmail.com      |29509916             |3    |37          |MOTORCYCLE SUSPENSION LIMITING SYSTEM      |
|ipdocket@thompsoncoburn.com |29216388             |3    |23          |P

<H4> Looking at one email address - jlonmur@roadrunner.com

In [25]:
df_all.select('emailaddresstext','title_length','inventiontitle').filter(col("emailaddresstext").contains("@roadrunner.com")).show(n=33,truncate=False)

+---------------------------+------------+----------------------------------------------------------------------+
|emailaddresstext           |title_length|inventiontitle                                                        |
+---------------------------+------------+----------------------------------------------------------------------+
|tfili@roadrunner.com       |32          |Pallet Rack Slide By Door System                                      |
|szell@roadrunner.com       |27          |Decorative animal tail seat                                           |
|augustinj@roadrunner.com   |22          |Huggabooh SnuggleBuddy                                                |
|buddydoyle@roadrunner.com  |36          |Kayak condos modular storage systems                                  |
|jlongmuir@roadrunner.com   |19          |SOLDERING APPARATUS                                                   |
|donerickson@roadrunner.com |22          |Butt refuse receptacle                        

<h4> Looking at "jlongmuir@roadrunner.com" data

In [26]:
df_jlongmuir=df_all.select('emailaddresstext','title_length','inventiontitle').filter(col("emailaddresstext").contains("jlongmuir@roadrunner.com"))
df_jlongmuir.show(n=33,truncate=False)

+------------------------+------------+----------------------------------------------------------------+
|emailaddresstext        |title_length|inventiontitle                                                  |
+------------------------+------------+----------------------------------------------------------------+
|jlongmuir@roadrunner.com|19          |SOLDERING APPARATUS                                             |
|jlongmuir@roadrunner.com|64          |Accessory Mount for Vehicle Heating and Air Conditioning Systems|
|jlongmuir@roadrunner.com|62          |Control Panel for Vehicle Heating and Air Conditioning Systems  |
|jlongmuir@roadrunner.com|17          |Hair Styling Tool                                               |
|jlongmuir@roadrunner.com|10          |Phone Case                                                      |
|jlongmuir@roadrunner.com|9           |Torch Tip                                                       |
|jlongmuir@roadrunner.com|17          |Hair Styling Too

In [27]:
#titles1=df_jlongmuir.select("inventiontitle").toPandas()['inventiontitle']
#list_titles=list(titles1)
print(list_titles[0:4])

NameError: name 'list_titles' is not defined

<h4> Looking at "wong.willy@yahoo.com" applications

In [28]:

df_wongwilly=df_all.select('emailaddresstext','title_length','inventiontitle').filter(col("emailaddresstext").contains("wong.willy@yahoo.com"))
df_wongwilly.show(n=33,truncate=False)

+--------------------+------------+--------------------------------------------------+
|emailaddresstext    |title_length|inventiontitle                                    |
+--------------------+------------+--------------------------------------------------+
|wong.willy@yahoo.com|24          |Mobile Phone Adapter Set                          |
|wong.willy@yahoo.com|26          |SQUARE SHAPE CEILING LIGHT                        |
|wong.willy@yahoo.com|11          |Wifi Camera                                       |
|wong.willy@yahoo.com|10          |TENT STOVE                                        |
|wong.willy@yahoo.com|17          |Hair Straightener                                 |
|wong.willy@yahoo.com|9           |Heel Grip                                         |
|wong.willy@yahoo.com|21          |Car-Shaped Bubble Toy                             |
|wong.willy@yahoo.com|50          |Square Shape Ceiling Light with Central Flat Panel|
|wong.willy@yahoo.com|23          |FORK OR 

<h4>  Need to get this package, sparkNLP

In [29]:
from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'sentence_transformers'

<h4> Detecting the language of the Invention titles

In [ ]:
!pip install spacy-langdetect

In [ ]:
import spacy
from spacy_langdetect import LanguageDetector

In [ ]:
!python -m venv .env
!source .env/bin/activate
!pip install -U spacy
!python -m spacy download en_core_web_lg

In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_news_lg-3.2.0/en_core_news_lg-3.2.0-py3-none-any.whl


In [ ]:
nlp = spacy.load('en_core_web_lg')  # 1
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
text_content = "Er lebt mit seinen Eltern und seiner Schwester in Berlin."
doc = nlp(text_content) 
detect_language = doc._.language 
print(detect_language)

In [ ]:
import spark-nlp

<h3> Tokenize the inventiontitles

In [ ]:
from pyspark.ml.feature import Tokenizer, Word2Vec
df = df_jlongmuir.select(['emailaddresstext','inventiontitle'])
newDF = df.distinct()
print(newDF.count())
newDF = newDF.withColumnRenamed('inventiontitle','text')
tokenizer = Tokenizer(outputCol="words")
tokenizer.setInputCol("text")

df_tokenizer = tokenizer.transform(newDF)
#df_tokenizer=df_tokenizer.withColumn('title_length',f.length('text'))
df_tokenizer.select("emailaddresstext","words").show(n=20,truncate=False)

<h4> Getting the needed packages for cosine_similarity

In [ ]:
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
a_file = df_tokenizer.select("words")
b_file = []
a_file.show(truncate=False)

<h4> Vectorizing the Invention titles with word2vec

In [ ]:
word2Vec = Word2Vec(vectorSize=500, seed=43, inputCol='words',outputCol="model")
word2Vec.setMaxIter(12)
print(word2Vec.getMaxIter())
#word2Vec.clear(word2Vec.maxIter)
doc = df_tokenizer.select('words')
#doc = df_text.select('words')
model = word2Vec.fit(doc)
#model.getMinCount()
model.getVectors().show(truncate=False,vertical=True)

In [ ]:
df_test = model.getVectors().select('vector')
print(df_test.count())
df_test.show(n=4)

In [ ]:
titles1=df_test.select("vector").toPandas()['vector']
print(titles1.shape)
list_titles=list(titles1)
print(list_titles[0:4])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
print(len(list_titles))
cosine_similarity([list_titles[0]],list_titles[1:])

In [ ]:
#invention_title = a_file.toPandas()
a_file.show(n=10,truncate=False)

<h4> Another example - wong.willy@yahoo.com

In [ ]:
from pyspark.ml.feature import Tokenizer, Word2Vec
df = df_wongwilly.select(['emailaddresstext','inventiontitle'])
newDF = df.distinct()
print(newDF.count())
newDF = newDF.withColumnRenamed('inventiontitle','text')
tokenizer = Tokenizer(outputCol="words")
tokenizer.setInputCol("text")

df_tokenizer = tokenizer.transform(newDF)
#df_tokenizer=df_tokenizer.withColumn('title_length',f.length('text'))
df_tokenizer.select("emailaddresstext","words").show(n=20,truncate=False)

##Vectorize the invention titles
word2Vec = Word2Vec(vectorSize=500, seed=43, inputCol='words',outputCol="model")
word2Vec.setMaxIter(12)
print(word2Vec.getMaxIter())
#word2Vec.clear(word2Vec.maxIter)
doc = df_tokenizer.select('words')
#doc = df_text.select('words')
model = word2Vec.fit(doc)
#model.getMinCount()
model.getVectors().show(truncate=False,vertical=True)
## Getting the vector column alone
df_test = model.getVectors().select('vector')
print(df_test.count())
df_test.show(n=4)

titles1=df_test.select("vector").toPandas()['vector']
print(titles1.shape)
list_titles=list(titles1)
print(list_titles[0:4])

print(len(list_titles))
cosine_similarity([list_titles[0]],list_titles[1:])


<h4> Looking at the effectivefilling data as another fraud signal

In [ ]:
df_pd = df_titles.toPandas()
df_pd.to_csv("/data/jupyter/Inventiontitles_2.csv",header=0)

In [ ]:
df_pd.head(n=10)

In [ ]:
df_titles.count()
df_titles.write.mode('overwrite').parquet("/data/jupyter/Inventiontitles.parquet")


In [ ]:
from pyspark.sql.functions import desc, col
df_1a = spark.sql("Select * from patent_data.opsg_email_address")
df_1 = df_1a.filter(col("emailaddresstext").isNotNull())
print(f'{df_1.count():,}')
print(df_1.printSchema())
df_1.groupby("emailaddresstext","applicationnumbertext").count().sort(desc("count")).show()

In [ ]:
##Matching emails with applications via the applicationnumbetext 
df_all = df_1.join(df_titles, on=['applicationnumbertext'] , how = 'left')

#print('========================')
print(f'{df_all.count():,}')
#print('====================')
df_all.show(n=5,truncate=False, vertical=True)

In [ ]:
##Getting the means and standard deviation of the character count of the title
df_spark1 = df_tokenizer.select('text','title_length')
df_spark1 = df_spark1.dropDuplicates(['text','title_length'])
print(f'{df_spark1.count():,}')
df_pd_1 = df_spark1.toPandas()
print(df_pd_1.title_length.mean())
print(df_pd_1.title_length.std())
df_pd_1.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
print(df_pd_1.shape)
print(df_pd_1.tail(n=10))
plt.title('Title length \n' 'n= 534,867 \n' 'mean = 43.78, std=33.78')
ax = plt.hist(df_pd_1['title_length'])
#ax.yaxis.set_major_formatter(mpl.ticker.StrMethodformatter('{x:,.0f}'))
#ax.set_yticklabels(['{:,}'.format(int(x)) for x in ax.get_yticks().tolist()])
plt.show()

In [ ]:
import pyspark.sql.functions as f
email = "chriochs@microsoft.com"
df_text = df_tokenizer.filter(col('emailaddresstext')==email)
# drop duplicates
df_text=df_text.dropDuplicates(['emailaddresstext','text','words'])
df_text=df_text.withColumn('title_length',f.length('text'))
print(df_text.count())
df_text.show(n=7,truncate=False)

<h3> applying word2vec to the tokenized titles

In [ ]:
pd_text = df_text.toPandas()
pd_text.head()

In [ ]:
!pip install spacy

In [ ]:
#!python -m spacy download en_core_web_lg
#!pip install spacy
import spacy
help(spacy.load)

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "the person wear red T-shirt",
    "this person is walking",
    "the boy wear red T-shirt"
    ])

print(embeddings)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")
#nlp = spacy.load("en_core_web_md")


doc1 = nlp(pd_text.iloc[0,0])
doc2 = nlp(pd_text.iloc[0,1])
doc3 = nlp(pd_text.iloc[0,2])


print(doc1.similarity(doc2)) 
print(doc1.similarity(doc3))
print(doc2.similarity(doc3)) 

In [ ]:
!pip install networkx
import networkx as nx

import matplotlib.pyplot as plt

In [ ]:
import np
nwords = 100
indexes = np.argpartition(dist,-(nwords+1))[-(nwords+1):] 
di = []
for counter in range(nwords_1):
    di.append((words[indexes[counter]],dist[indexes[counter]], labels[indexes[counter]]))
print(di[2])    

In [ ]:
df_email_time=df_all.groupby('emailaddresstext','effectivefilingdate').count().sort(desc("count"))
df_pd_email_time = df_email_time.toPandas()

import pandas as pd
import matplotlib.pyplot as plt

df_x = df_pd_email_time[df_pd_email_time['count'] > 3]
df_x = df_x[df_x['count'] < 40 ]
print(df_x.shape)
print(df_x.tail(n=60))
plt.hist(df_x['count'])



In [ ]:
df_1.groupby("emailaddresstext").count().sort(desc("count")).show(n=50,truncate=False)

<h4> Suhee application numbers

In [ ]:
df_2 = spark.sql("Select * from patent_data.suhee_applications")
print(f'{df_2.count():,}')
print(df_2.printSchema())
print(df_2.show())

In [ ]:
##Seeing email matches with Suhee park
df_3 = df_2.join(df_1 , on=['applicationnumbertext'] , how = 'left').show(n=50,truncate=False)


<h4> Joining the Suhee data to the application data

In [ ]:
df_4 = df_2.join(df_0, on=['applicationnumbertext'] , how = 'left')
print(f'{df_4.count():,}')
df_4.show(n=5,truncate=False, vertical=True)

In [ ]:
df_1a = df_1.filter(df_1.emailaddresstext=='info@24ipusa.com')
print(f'{df_1a.count():,}')
df_1a.show(n=5)

In [ ]:
##Joining the email address info@24ipusa.com by applicationnumbertext to emailaddresstext
df_1b = df_1a.join(df_0, on=['applicationnumbertext'] , how = 'left')
print(f'{df_1b.count():,}')
df_1b.show(n=5,truncate=False,vertical=True)

In [ ]:
email = "chriochs@microsoft.com"
df_1aa = df_1.filter(df_1.emailaddresstext==email)
df_1ab = df_1aa.join(df_0, on=['applicationnumbertext'] , how = 'left')
print(f'{df_1ab.count():,}')
df_1ab.select('emailaddresstext','applicationnumbertext','effectivefilingdate','inventiontitle','applicationstatusdescriptiontext').sort(desc("effectivefilingdate")).show(n=130,truncate=False, vertical=True)
#df_1ab.show(n=15,truncate=False, vertical=True)

In [ ]:
##Another join of application data to emailaddresstext

df_1aa = df_1.filter(df_1.emailaddresstext=='sughrue@sughrue.com')
df_1ab = df_1aa.join(df_0, on=['applicationnumbertext'] , how = 'left')
print(f'{df_1ab.count():,}')
df_1ab.show(n=15,truncate=False, vertical=True)

In [ ]:
##Another join of application data to emailaddresstext

df_1ac = df_1.filter(df_1.emailaddresstext=='USPTO@sughrue.com')
df_1ad = df_1ac.join(df_0, on=['applicationnumbertext'] , how = 'left')
print(f'{df_1ad.count():,}')
df_1ad.show(n=15,truncate=False, vertical=True)

<h3> Another email address to look at

In [ ]:
email='jarmstrong@oliff.com' 

df_1ac = df_1.filter(df_1.emailaddresstext==email)
df_1ad = df_1ac.join(df_0, on=['applicationnumbertext'] , how = 'left')
print(f'{df_1ad.count():,}')
df_1ad.show(n=15,truncate=False, vertical=True)

<h4> Looking at the Abandoned records

In [ ]:
from pyspark.sql.functions import lit, col

In [ ]:
df_aband=df_0.filter(col("applicationstatusdescriptiontext").like("%Abandoned%"))
print('Record counts = ', f'{df_aband.count():,}')
df_aband.groupby("applicationnumbertext").count().sort(desc("count")).show()

<h4> Merging the abandoned data with email by applicationnumbertext

In [ ]:
df_1ad = df_aband.join(df_1, on=['applicationnumbertext'] , how = 'left')
print(f'{df_1ad.count():,}')
df_1ad.show(n=15,truncate=False, vertical=True)

<h4> Getting rid of the null(s) emails

In [ ]:
df_nonulls_email = df_1ad.filter(col("emailaddresstext").isNotNull())
print(f'{df_nonulls_email.count():,}')
df_nonulls_email.show(n=15,truncate=False, vertical=True)

<h3> Counts of email address 

In [ ]:
df_nonulls_email.groupby("emailaddresstext").count().sort(desc("count")).show(n=150, truncate=False)

<h4> Finding emails and application data with questionable data

In [ ]:
email='%sinorica@%'
df_1ac=df_1.filter(col("emailaddresstext").like(email))

df_1ad = df_1ac.join(df_0, on=['applicationnumbertext'] , how = 'left')
print(f'{df_1ad.count():,}')
df_1ad.show(n=15,truncate=False, vertical=True)

<h4> Fraud found

In [ ]:
email='XOXOAB@YAHOO.COM' 

df_1ac = df_1.filter(df_1.emailaddresstext==email)
df_1ad = df_1ac.join(df_0, on=['applicationnumbertext'] , how = 'left')
print(f'{df_1ad.count():,}')
df_1ad.show(n=15,truncate=False, vertical=True)

In [ ]:
df_1ad.select('emailaddresstext','applicationnumbertext','effectivefilingdate','inventiontitle').sort(desc("effectivefilingdate")).show(n=15,truncate=False, vertical=True)

In [ ]:
email='Stillrolling25@gmail.com' 

df_1ac = df_1.filter(df_1.emailaddresstext==email)
df_1ad = df_1ac.join(df_0, on=['applicationnumbertext'] , how = 'left')
print(f'{df_1ad.count():,}')
df_1ad.show(n=15,truncate=False, vertical=True)

In [ ]:
df_1ad.select('emailaddresstext','applicationnumbertext','effectivefilingdate','inventiontitle').sort(desc("effectivefilingdate")).show(n=15,truncate=False, vertical=True)

In [ ]:
df_1ad.select('emailaddresstext','effectivefilingdate').sort(desc("effectivefilingdate")).show(n=50,truncate=False)